# b06705048_hw3

## Load package and data
Load packages and data, and then standardize all feature values.
Now, we have data：

## Q1.1
1. Clean up the dataset 
2. create x_train, y_train, x_test, y_test for training feature, training value, test feature, test label. 
3. All of these variables should be numpy arrays. 
4. Provide summary statistics for your training and test datasets

In [1]:
import pandas as pd
import numpy as np
import pickle
import math

In [2]:
train=pd.read_csv("train.csv",header=None)
test=pd.read_csv("test.csv",header=None)
print(train.shape)
print(test.shape)

(32561, 15)
(16281, 15)


In [3]:
all = ["age","workclass","fnlwgt","education","education-num","marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week","native-country","income"]
train.columns = all
test.columns = train.columns

In [4]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [6]:
# data cleaning
##  You should remove all rows with missing values. 
for i in train.columns:
    train[i] = train[i].replace(' ?', value = np.nan)
    test[i] = test[i].replace(' ?', value = np.nan)
test.dropna(inplace = True)
train.dropna(inplace = True)

In [7]:
print(train.shape)
print(test.shape)

(30162, 15)
(15060, 15)


In [8]:
##  Include a particular feature value only if this unique value appears more than ten times in the training data.
text = ["workclass","education","marital-status","occupation","relationship","race","sex","native-country"]
for i in text:
    print(train[i].value_counts()[train[i].value_counts() < 10])
# find：Armed-Forces /  Holand-Netherlands  will be removed

Series([], Name: workclass, dtype: int64)
Series([], Name: education, dtype: int64)
Series([], Name: marital-status, dtype: int64)
 Armed-Forces    9
Name: occupation, dtype: int64
Series([], Name: relationship, dtype: int64)
Series([], Name: race, dtype: int64)
Series([], Name: sex, dtype: int64)
 Holand-Netherlands    1
Name: native-country, dtype: int64


In [9]:
## All features with discrete-values 
## for example, native-country and workclass should be converted to "1-of-K" encoding.
train = pd.concat([train.drop(text,axis=1),pd.get_dummies(train[text])],axis=1)
test = pd.concat([test.drop(text,axis=1),pd.get_dummies(test[text])],axis=1)

In [10]:
print(train.shape)
print(test.shape)
for i in range(len(train.columns)):
    if train.columns[i] not in test.columns:
        print(train.columns[i],"not in test")
        #test.insert(i,train.columns[i],0)
for i in range(len(test.columns)):
    if test.columns[i] not in train.columns:
        print(test.columns[i],"not in train")

(30162, 105)
(15060, 104)
native-country_ Holand-Netherlands not in test


In [11]:
train.drop(["occupation_ Armed-Forces","native-country_ Holand-Netherlands"],axis=1,inplace=True)
test.drop(["occupation_ Armed-Forces"],axis=1,inplace=True)

In [12]:
print(train.shape)
print(test.shape)

(30162, 103)
(15060, 103)


In [13]:
mapping={" >50K":1," <=50K":0}
y_train=pd.DataFrame([])
y_train=train["income"].map(mapping)
mapping2={" >50K.":1," <=50K.":0}
y_test=pd.DataFrame([])
y_test=test["income"].map(mapping2)
x_train=train.drop("income",axis=1)
x_test=test.drop("income",axis=1)

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(30162, 102)
(30162,)
(15060, 102)
(15060,)


In [15]:
## summary statistics for your training and test datasets 
x_train.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,...,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238,0.031265,0.068530,0.738877,0.035608,...,0.001127,0.003614,0.000365,0.002354,0.001392,0.000564,0.000597,0.911876,0.002122,0.000530
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984,0.174035,0.252657,0.439254,0.185313,...,0.033556,0.060007,0.019094,0.048461,0.037291,0.023734,0.024422,0.283480,0.046016,0.023026
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.176272e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,47.000000,2.376285e+05,13.000000,0.000000,0.000000,45.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
y_train.describe()

count    30162.000000
mean         0.248922
std          0.432396
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: income, dtype: float64

In [17]:
x_test.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
count,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,...,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,1.896164e+05,10.112749,1120.301594,89.041899,40.951594,0.030744,0.068592,0.731806,0.037981,...,0.001859,0.004382,0.000598,0.001992,0.000863,0.000797,0.000531,0.915538,0.001262,0.000465
std,13.380676,1.056150e+05,2.558727,7703.181842,406.283245,12.062831,0.172628,0.252768,0.443034,0.191158,...,0.043080,0.066057,0.024440,0.044589,0.029369,0.028218,0.023043,0.278089,0.035498,0.021555
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.166550e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.779550e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,48.000000,2.385888e+05,13.000000,0.000000,0.000000,45.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
y_test.describe()

count    15060.000000
mean         0.245684
std          0.430506
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: income, dtype: float64

In [19]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

## Q1.2 & Q1.3
#### Derive the gradient and hessian matrix for the new E(w).
#### Create your mylogistic_l2 class.

In [20]:
# sample use:
# logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
# logic1.fit(X_train, Y_train)
# ypred = logic1.predict(X_test)


class mylogistic_l2:
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept

    
    def fit(self,X_train,Y_train):
        if self.add_intercept == True:
            X_train = np.concatenate((np.ones((X_train.shape[0],1)),X_train), axis = 1)
        self.w = np.dot(np.dot(np.linalg.inv(np.dot(X_train.T,X_train)+np.identity(X_train.shape[1])*self.reg_vec.mean()),X_train.T),Y_train)
        error = [0]
        
        for i in range(1,self.max_iter):
            z = np.dot(X_train, np.transpose(self.w))
            
            ## Q1.2 
            #### Derive the gradient and hessian matrix for the new E(w).
            ### y: predicted value after sigmoid function
            y = 1 / (1.0 + np.exp(-z))
           
            reg_term=(1/2)*np.dot(np.dot(self.w.T,np.diag(self.reg_vec.ravel())),self.w)
            
            cross_entropy = (-1 * (np.dot(np.squeeze(Y_train), np.log(y)) + np.dot((1 - np.squeeze(Y_train)), np.log(1 - y)))+reg_term)/X_train.shape[0]
            error.append(cross_entropy)
            
            ### w_grad of the E(w)
            w_grad = X_train.T.dot(y-Y_train)+self.reg_vec*self.w
            
            ### hession matrix for the E(w)
            ds = y*(1-y)
            D = np.diag(ds.ravel()) 
            H =  X_train.T.dot( D.dot(X_train) )+np.identity(self.reg_vec.shape[0])*self.reg_vec
            #update w
            self.w = self.w -np.linalg.inv(H).dot(w_grad)
            
            ### update rule:
            if i > 1:
                if error[i-1]-error[i]<self.tol:
                    break
                    
    def predict(self,X_test):
        if self.add_intercept == True:
            X_test = np.concatenate((np.ones((X_test.shape[0],1)),X_test), axis = 1)
        z = np.dot(X_test,self.w) 
        y = 1 / (1.0 + np.exp(-z))
        y_ = np.where(y>= 0.5, 1, 0)
        return y_, self.w

#### Case 1: 
lambda = 1 for all coefficients

In [21]:
a = np.ones(x_train.shape[1] + 1)
logic1 = mylogistic_l2(max_iter = 100, add_intercept = True, reg_vec = a, tol = 1e-5)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("print the learned w","\n",ypred[1])
print("print test accuracy","\n",(ypred[0] == y_test).sum()/len(y_test))

print the learned w 
 [-3.33374540e+00  2.48591297e-02  7.26281835e-07  1.85806163e-01
  3.16590245e-04  6.38679434e-04  2.90215841e-02  1.89694411e-01
 -4.97719501e-01 -3.10975202e-01 -1.29181491e-01 -7.94111556e-01
 -6.18745003e-01 -1.17270706e+00 -4.52249247e-01 -5.44907403e-01
 -3.82721832e-01 -9.99810289e-02 -2.62922225e-01 -6.29791388e-01
 -4.96565839e-01 -3.08326937e-01 -1.27584811e-01  1.28534378e-01
  5.87953355e-01 -2.50481306e-01  2.99991415e-01 -1.37579993e+00
  6.82097029e-01 -1.00989638e-01 -1.00975740e+00  1.19612258e+00
  8.38966933e-01 -9.56673066e-01 -1.50523218e+00 -1.08232391e+00
 -8.14848363e-01 -8.78492150e-02 -2.46913576e-02  7.15174653e-01
 -1.06504054e+00 -7.73348089e-01 -3.53722827e-01 -9.03671285e-01
 -1.69395623e+00  4.29670534e-01  4.99488476e-01  2.05962313e-01
  5.68478621e-01 -1.78947443e-01 -5.76254828e-01 -3.75781253e-01
 -1.12608337e+00 -1.50192087e+00 -5.01524915e-01  7.47819835e-01
 -1.02439618e+00 -2.92668732e-01 -6.35195701e-01 -8.96890177e-01
 -4

#### Case 2: 
lambda = 1 for all but the intercept, no regularization for incercept term.

In [22]:
a = np.ones(x_train.shape[1] + 1)
a[0] = 0  #set the weight of intercept to 0
logic1 = mylogistic_l2(max_iter = 100,add_intercept = True,reg_vec = a,tol=1e-5)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("print the learned w","\n",ypred[1])
print("print test accuracy","\n",(ypred[0] == y_test).sum()/len(y_test))

print the learned w 
 [-8.88722778e+00  2.54336822e-02  7.50706804e-07  2.95324923e-01
  3.17024478e-04  6.39652126e-04  2.94914512e-02  7.05717318e-01
  1.78026502e-02  2.09126595e-01  3.82731145e-01 -2.79821710e-01
 -1.04552949e-01 -9.31003049e-01  9.08118759e-02 -1.06210390e-01
 -5.68864876e-02  7.06129017e-01  5.36014978e-01  1.16361326e-01
  1.37527872e-01 -4.01951495e-01 -1.13370201e-01 -7.43801938e-02
  6.79669927e-02 -1.95742295e-02 -1.15929725e-02 -1.15969699e+00
  2.67049827e-01  2.18010722e-02 -5.26538170e-01  1.61452756e+00
  1.36750998e+00 -4.92456773e-01 -1.01532721e+00 -6.05766406e-01
 -3.41948994e-01  1.64109182e-01  2.28422252e-01  9.64856017e-01
 -8.17967488e-01 -5.20782272e-01 -9.91243553e-02 -6.49283758e-01
 -1.55300394e+00  6.78427763e-01  7.51030496e-01  4.55411798e-01
  8.18714334e-01  7.31938650e-02 -4.21151177e-02  1.99456835e-01
 -5.83542676e-01 -9.36996676e-01  7.53014460e-02  1.28789619e+00
 -3.72048376e-01  3.94360616e-01  4.30693947e-02 -2.61235809e-01
  1

#### Case 3: 
lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.

In [23]:
#dummy variable start from six
x_train[:,6:]  

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [24]:
a = np.ones(x_train.shape[1] + 1)
a[0] = 0 #set the weight of intercept to 0
a[6:] = 0.5
logic1 = mylogistic_l2(max_iter =100,add_intercept=True,reg_vec =a,tol=1e-5)
logic1.fit(x_train, y_train)
ypred = logic1.predict(x_test)
print("print the learned w","\n",ypred[1])
print("print test accuracy","\n",(ypred[0] == y_test).sum()/len(y_test))

print the learned w 
 [-9.31135249e+00  2.54757380e-02  7.51944447e-07  3.19092334e-01
  3.17319918e-04  6.40115467e-04  2.95136416e-02  7.66990129e-01
  7.62915572e-02  2.68338062e-01  4.43127805e-01 -2.20821183e-01
 -4.63421363e-02 -1.28758423e+00  2.18992444e-01 -2.26135094e-03
  2.18816221e-02  9.74981515e-01  7.54034090e-01  2.92224599e-01
  2.93070047e-01 -4.19612047e-01 -1.04346432e-01 -1.12963353e-01
 -3.78241652e-02  3.76517948e-02 -7.30515953e-02 -2.08394381e+00
  1.86338326e-01  5.48283203e-02 -5.72016479e-01  1.82571076e+00
  1.39641686e+00 -5.47067030e-01 -1.05936555e+00 -6.55642109e-01
 -3.88036449e-01  2.36234568e-01  3.00253587e-01  1.03825742e+00
 -7.52670195e-01 -4.53424296e-01 -2.69070976e-02 -5.82347486e-01
 -2.00233663e+00  7.51037287e-01  8.27361959e-01  5.28329988e-01
  8.95058089e-01  1.45241736e-01 -8.34438876e-02  2.32698984e-01
 -5.92702310e-01 -9.22764380e-01  1.11240494e-01  1.25497110e+00
 -3.83664223e-01  4.13058389e-01  4.13741722e-02 -2.63868110e-01
  1

# Q1.4

conduct grid search with the constraint that a1=a2 . Record the best value a∗1 and a∗2 .

In [25]:
# Further split the training data into subtraining (90%) and tuning (10%) 
# to search for the best hyperparameters. 

all = len(x_train)
valid = int(math.floor(all * 0.1)) # Round down

X_valid = x_train[0:valid]
Y_valid = y_train[0:valid]
X_train = x_train[valid:]
Y_train = y_train[valid:]

print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(27146, 102)
(27146,)
(3016, 102)
(3016,)


In [26]:
# Step 1:
# choose a set of grids among a reasonable range.
# evenly dispersed points (start, stop, num)
find = np.linspace(0.01,100, num=10) 
print(find)

[1.000e-02 1.112e+01 2.223e+01 3.334e+01 4.445e+01 5.556e+01 6.667e+01
 7.778e+01 8.889e+01 1.000e+02]


In [27]:
# Step 2:
# Conduct grid search with the constraint that  a1=a2
# Record the best value  a∗1  and  a∗2 .
a = np.ones(x_train.shape[1] + 1)
a[0] = 0 #set the weight of intercept to 0
collect = []
for i in range(len(find)):
    a = np.ones(x_train.shape[1] + 1)
    a[0] = 0  
    a[1:]= find[i]
    logic1 = mylogistic_l2(max_iter =100,add_intercept=True,reg_vec =a,tol=1e-5)
    logic1.fit(X_train, Y_train)
    ypred = logic1.predict(X_valid)
    ans = (ypred[0]==Y_valid).sum()
    collect.append(ans/len(Y_valid))

In [28]:
ans = np.argmax(collect)
minnumber = find[ans]
# when a1=a2, the best regularization coefficients
# the best value  of a∗1 and a∗2
minnumber 

0.01

In [29]:
# Step 3:
# Fix a1=a∗1 , and search a2 for the best value
# call the result the new a∗2 .
a = np.ones(x_train.shape[1] + 1)
a[0] = 0 
collect = []
for i in range(len(find)):
    a = np.ones(x_train.shape[1] + 1)
    a[0] = 0  
    a[:6] = 0.01 ##Fix a1=a∗1 
    a[6:] = find[i]
    logic1 = mylogistic_l2(max_iter =100,add_intercept=True,reg_vec =a,tol=1e-5)
    logic1.fit(X_train, Y_train)
    ypred = logic1.predict(X_valid)
    ans=(ypred[0]==Y_valid).sum()
    collect.append(ans/len(Y_valid))

In [30]:
ans=np.argmax(collect)
minnumber=find[ans]
# when a1=a*1, the best a2
minnumber  

0.01

In [31]:
# Step 4:
# Fix a2 = a∗2 , and search a1 for the best value.v
a = np.ones(x_train.shape[1] + 1)
a[0] = 0 
collect = []
for i in range(len(find)):
    a = np.ones(x_train.shape[1] + 1)
    a[0] = 0  
    a[6:] = 0.01 ##Fix a2=a∗2
    a[:6] = find[i]
    logic1 = mylogistic_l2(max_iter =100,add_intercept=True,reg_vec =a,tol=1e-5)
    logic1.fit(X_train, Y_train)
    ypred = logic1.predict(X_valid)
    ans=(ypred[0]==Y_valid).sum()
    collect.append(ans/len(Y_valid))

In [32]:
ans = np.argmax(collect)
minnumber = find[ans]
# when a2=a*2, the best a1
minnumber  

0.01

In [33]:
# Step 5:
# Report the selected a1 and a2 
# the best choice is ： a1=a∗1 and a2=a∗2

In [34]:
# Step 6:
# Train a model using the selected hyper-parameters
# and report the test accuracy.
a[0] = 0  
a[1:] = 0.01
logic1 = mylogistic_l2(max_iter =100,add_intercept=True,reg_vec =a,tol=1e-5)
logic1.fit(x_train,y_train)
ypred = logic1.predict(x_test)

# print test accuracy
acc = (ypred[0]==y_test).sum()/len(y_test)
print("test accuracy",acc)

test accuracy 0.847808764940239


# Q1.5

Use sklearn.linear_model.LogisticRegression to train and test the model (including hyper-parameter tuning). Compare the estimated parameters and test accuracy with those from your own models.

In [67]:
# with sklearn.linear_model.LogisticRegression
from sklearn.linear_model import LogisticRegression
r=[]                    
for alpha in find:
    clf = LogisticRegression(penalty="l2",C=alpha).fit(X_train, Y_train)
    ypred=clf.predict(X_valid)
    acc=(ypred==Y_valid).sum()/len(Y_valid)
    r.append(acc)

In [68]:
a = np.argmax(r)
ans = find[a]
ans

0.01

In [69]:
clf = LogisticRegression(penalty="l2",C=0.01).fit(x_train, y_train)
ypred = clf.predict(x_test)
acc = (ypred==y_test).sum()/len(ypred)

# Compare the test accuracy 
print("test accuracy of sklearn model",acc)

test accuracy of sklearn model 0.7944887118193891


D:\lalami\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [70]:
#with my own model
a=np.ones(x_train.shape[1] + 1)
a[0]=0  
a[1:]=0.01
logic1 = mylogistic_l2(max_iter =100,add_intercept=True,reg_vec =a,tol=1e-5)
logic1.fit(x_train,y_train)
ypred = logic1.predict(x_test)
acc=(ypred[0]==y_test).sum()/len(y_test)
print("test accuracy of my model",acc)

test accuracy of my model 0.847808764940239


In [76]:
# Compare the estimated parameters
print("the estimated parameters of sklearn model")
print(clf.coef_)
print(clf.intercept_)

the estimated parameters of sklearn model
[[ 2.94298860e-03 -3.95344524e-06 -3.13044246e-03  3.22502109e-04
   8.46122953e-04 -1.74357732e-02  6.81470567e-04  2.10670636e-04
  -9.13631093e-03  1.63903756e-03 -1.43066354e-04 -1.29033740e-04
  -2.82392735e-05 -1.09207798e-03 -1.61307198e-03 -5.07654306e-04
  -1.94554250e-04 -3.92017121e-04 -7.67528493e-04 -6.13915782e-04
  -1.47414353e-04 -1.88633982e-04  4.29883752e-03  1.04929331e-03
  -7.12470553e-03  2.69767891e-03 -7.97277404e-05  1.27687044e-03
  -3.50685021e-03 -4.85491377e-03  3.40058531e-05  1.60791403e-02
  -4.64807146e-04 -1.54294366e-02 -1.23600447e-03 -1.03345570e-03
  -3.58066005e-03 -1.25493415e-03  4.94104717e-03 -1.11393802e-03
  -1.88296406e-03 -1.85296922e-03 -5.10039763e-03 -2.53043438e-04
   3.79493564e-03  3.06933691e-04 -3.93874062e-04  1.74928484e-04
  -6.81568990e-04  1.44363598e-02 -9.18322629e-03 -1.36955170e-03
  -8.18620036e-03 -4.44481910e-03  1.84196613e-03 -4.15476227e-04
  -1.36577238e-04 -2.34474634e-03 

In [77]:
print("the estimated parameters of my model")
ypred[1]

the estimated parameters of my model


array([-1.14761639e+01,  2.55402069e-02,  7.54179075e-07,  3.99000844e-01,
        3.18079304e-04,  6.40833979e-04,  2.95698399e-02,  1.18875412e+00,
        4.92451904e-01,  6.84930795e-01,  8.61390951e-01,  1.95387325e-01,
        3.70139269e-01, -3.79305437e+00,  6.42711568e-01,  3.40923489e-01,
        2.87599360e-01,  1.80393087e+00,  1.45363314e+00,  8.78329015e-01,
        8.06064563e-01, -4.81427165e-01, -8.29135223e-02, -2.51104361e-01,
       -4.13705383e-01,  2.19646908e-01, -2.90687139e-01, -4.96327039e+00,
       -1.06146525e-01,  1.56415569e-01, -6.32068788e-01,  2.09585250e+00,
        1.43979783e+00, -6.18496764e-01, -1.11726140e+00, -7.20624634e-01,
       -4.47198738e-01,  9.58865053e-01,  1.02259679e+00,  1.76270281e+00,
       -3.59538839e-02,  2.65170841e-01,  6.99095129e-01,  1.35348361e-01,
       -3.04316652e+00,  1.47439297e+00,  1.55517451e+00,  1.25201407e+00,
        1.62275162e+00,  8.68137958e-01, -1.42337820e-01,  2.77065751e-01,
       -6.00456094e-01, -

### Compared two model
the estimated parameters of my own model is bigger but test accuracy is higher than sklearn.linear_model.LogisticRegression
